In [1]:
from qiskit import QuantumCircuit, qasm3
from quantum_chip import QuantumChip
from hardware import HardwareParams
from qrmove_compiler import QRMoveCompiler
from utils import get_quantum_circuit
import math
from func_timeout import func_timeout, FunctionTimedOut


# Default Settings
# quantum_alg: bv
# MRP_CP_ratio: 10
# extra_qubit_ratio: 15%
# qubit_num: 30


def experiment(quantum_alg, qubit_num, extra_qubit_num, MRP_CP_ratio):
    """
    quantum_alg: 量子算法
    qubit_num: 量子比特数量
    extra_qubit_ratio: 额外比特比例
    MRP_CP_ratio: MRP/CP 比例
    """
    try:
        quantum_circuit: QuantumCircuit = get_quantum_circuit(quantum_alg, qubit_num)
        quantum_chip = QuantumChip("square", 50)
        hardware_param = HardwareParams(
            time_1q=300.0,  # 单比特门时间
            time_2q=300.0,  # 双比特门时间
            time_meas=300 * MRP_CP_ratio,  # 测量时间
            time_reset=0,  # 重置时间
        )
        qmc = QRMoveCompiler(
            quantum_circuit,
            quantum_chip,
            hardware_param,
            math.ceil(extra_qubit_num),
        )
        (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        ) = qmc.compile_program()

        # print(
        #     f"-- 运行结果: start_depth {start_depth}, start_width {start_width}, qr_map_depth {qr_map_depth}, qr_map_width {qr_map_width}, qr_move_depth {qr_move_depth}, qr_move_width {qr_move_width} --"
        # )

        return (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        )

    except Exception as e:
        print(f"Experiment failed with error: {e}")
        return 0, 0, 0, 0, 0, 0

In [24]:
# Default Settings
# quantum_alg: bv
# MRP_CP_ratio: 10
# extra_qubit_ratio: 15%
# qubit_num: 30
# bv qaoa mod mul qram rd sym xor
for alg in ["xor"]:
    # for alg in ["bv", "qaoa", "mod", "mul", "qram", "rd", "sym", "xor"]:
    for extra_num in [1]:
        qubit_num = 30
        mrp_ratio = 10
        (
            start_depth,
            start_width,
            qr_map_depth,
            qr_map_width,
            qr_move_depth,
            qr_move_width,
        ) = experiment(alg, qubit_num, extra_num, mrp_ratio)
        print(f"alg: {alg}, extra_num: {extra_num}")
        print(
            f"-- 运行结果: start_depth {start_depth}, start_width {start_width}, qr_map_depth {qr_map_depth}, qr_map_width {qr_map_width}, qr_move_depth {qr_move_depth}, qr_move_width {qr_move_width} --"
        )

alg: xor, extra_num: 1
-- 运行结果: start_depth 14, start_width 30, qr_map_depth 183, qr_map_width 4, qr_move_depth 183, qr_move_width 4 --


In [ ]:
def experiment_with_timeout(
    quantum_alg, qubit_num, extra_qubit_num, MRP_CP_ratio, timeout=60 * 30
):
    """
    带超时控制的实验函数
    timeout: 超时时间（秒），默认5分钟
    """
    try:
        result = func_timeout(
            timeout,
            experiment,
            args=(quantum_alg, qubit_num, extra_qubit_num, MRP_CP_ratio),
        )
        return result
    except FunctionTimedOut:
        print(f"Experiment timed out after {timeout} seconds")
        return 0, 0, 0, 0, 0, 0


def experiment_var_extra_ratio_alg():
    import csv
    import os

    # 创建结果文件夹
    os.makedirs("results", exist_ok=True)

    # 创建CSV文件并写入表头
    csv_file = "results/experiment_results.csv"
    with open(csv_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(
            [
                "Algorithm",
                "Extra_Ratio",
                "Qubit_Num",
                "Avg_Start_Depth",
                "Avg_Start_Width",
                "Avg_QR_Map_Depth",
                "Avg_QR_Map_Width",
                "Avg_QR_Move_Depth",
                "Avg_QR_Move_Width",
                "Valid_Experiments",
            ]
        )

    # 存储每个参数组合的任务结果
    all_tasks = []

    # 创建所有参数组合
    for alg in ["bv", "vqa", "qaoa", "qft", "mul", "rd", "xor"]:
        for extra_ratio in [0.1, 0.2, 0.3]:
            qubit_num = 30
            all_tasks.append((alg, extra_ratio, qubit_num))

    # 为每个参数组合执行50次实验
    def run_experiments_for_params(alg, extra_ratio, qubit_num):
        results = []
        for _ in range(50):
            (
                start_depth,
                start_width,
                qr_map_depth,
                qr_map_width,
                qr_move_depth,
                qr_move_width,
            ) = experiment_with_timeout(alg, qubit_num, extra_ratio * qubit_num, 10)

            # 如果实验成功（不是全零结果），则添加到结果列表中
            if not (
                start_depth == 0
                and start_width == 0
                and qr_map_depth == 0
                and qr_map_width == 0
                and qr_move_depth == 0
                and qr_move_width == 0
            ):
                results.append(
                    (
                        start_depth,
                        start_width,
                        qr_map_depth,
                        qr_map_width,
                        qr_move_depth,
                        qr_move_width,
                    )
                )
        return alg, extra_ratio, qubit_num, results

    # 使用多线程并发执行所有实验
    results_dict = {}
    import threading
    from concurrent.futures import ThreadPoolExecutor, as_completed

    # 降低线程数以避免资源竞争
    with ThreadPoolExecutor(max_workers=16) as executor:
        # 提交所有任务
        future_to_params = {
            executor.submit(run_experiments_for_params, alg, extra_ratio, qubit_num): (
                alg,
                extra_ratio,
                qubit_num,
            )
            for alg, extra_ratio, qubit_num in all_tasks
        }

        # 收集结果
        for future in as_completed(future_to_params):
            alg, extra_ratio, qubit_num, valid_results = future.result()
            results_dict[(alg, extra_ratio)] = valid_results

    # 处理和打印结果，并将均值数据写入文件
    with open(csv_file, mode="a", newline="") as file:
        writer = csv.writer(file)

        for alg in ["bv", "vqa", "qaoa", "qft", "mul", "rd", "xor"]:
            for extra_ratio in [0.1, 0.2, 0.3]:
                qubit_num = 30
                valid_results = results_dict.get((alg, extra_ratio), [])

                print(
                    f"-- 参数: {alg} 算法, 额外比特比例 {extra_ratio}, 量子比特数量 {qubit_num} --"
                )

                # 计算平均值
                if valid_results:
                    avg_start_depth = sum(r[0] for r in valid_results) / len(
                        valid_results
                    )
                    avg_start_width = sum(r[1] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_map_depth = sum(r[2] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_map_width = sum(r[3] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_move_depth = sum(r[4] for r in valid_results) / len(
                        valid_results
                    )
                    avg_qr_move_width = sum(r[5] for r in valid_results) / len(
                        valid_results
                    )

                    print(
                        f"平均值: {avg_start_depth}, {avg_start_width}, {avg_qr_map_depth}, {avg_qr_map_width}, {avg_qr_move_depth}, {avg_qr_move_width}"
                    )
                    print(f"有效实验次数: {len(valid_results)}")

                    # 将结果写入CSV文件
                    writer.writerow(
                        [
                            alg,
                            extra_ratio,
                            qubit_num,
                            avg_start_depth,
                            avg_start_width,
                            avg_qr_map_depth,
                            avg_qr_map_width,
                            avg_qr_move_depth,
                            avg_qr_move_width,
                            len(valid_results),
                        ]
                    )
                else:
                    print("所有实验都失败了，无法计算平均值")
                    # 即使实验失败，也记录到文件中
                    writer.writerow([alg, extra_ratio, qubit_num, 0, 0, 0, 0, 0, 0, 0])


experiment_var_extra_ratio_alg()